In [1]:
from comet_ml import Experiment

In [2]:
import numpy as np 
import shap
import pickle
import json
import time
import json
import pandas as pd
import torch
import math
import os
import torch.nn as nn
import torch.nn.functional as F
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, roc_curve, auc
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import sys
sys.path.append('../ift6758/visualizations/')

from question_5_plots import *

In [3]:
df = pd.read_csv('final_df.csv')

In [4]:
df_test = df[df['season'] == 20192020]
df_test = df[df['season_type'] == 'R']

In [5]:
# xgb = pickle.load(open("random_forest", 'rb'))
# rf = pickle.load(open("random_forest", 'rb'))
log1 = pickle.load(open("LogisticRegression_Distance", 'rb'))
log2 = pickle.load(open("LogisticRegression_Angle", 'rb'))
log3 = pickle.load(open("LogisticRegression_Distance+Angle", 'rb'))

In [6]:
old_feat = ['game_id', 'season', 'season_type', 'event_id', 'home_team',
       'away_team', 'home_team_side_1st_period', 'attacking_team',
       'attacking_player', 'goalie', 'period', 'period_time', 'goal_ind',
       'shot_ind', 'x_coordinates', 'y_coordinates', 'shot_type', 'empty_net',
       'strength', 'gwg', 'previous_event_x_coordinates',
       'previous_event_y_coordinates', 'previous_event_period_time',
       'previous_event_type', 'previous_event_period', 'previous_attacking_team',
       'attacking_team_side']

df_feat = df.iloc[:,:60].columns.tolist()

#features extracted in question 4
new_feat = [feat for feat in df_feat if feat not in old_feat]

In [7]:
X_rf = df_test[new_feat]
X_lr_D = df_test['distance_from_net'].values.reshape(-1,1)
X_lr_A = df_test['angle_from_net'].abs().values.reshape(-1,1)
X_lr_DA = df_test[['distance_from_net','angle_from_net']]
y = df_test['goal_ind']
print(X_lr_D)

[[34.53]
 [10.  ]
 [14.04]
 ...
 [24.02]
 [24.19]
 [19.1 ]]


In [10]:
preds_rf = rf.predict(X_rf)
# preds_lr_d = log1.predict(X_lr_D)
# preds_lr_a = log2.predict(X_lr_A)
preds_lr_da = log3.predict(X_lr_DA)

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- shot_type_Wrist Shot
Feature names seen at fit time, yet now missing:
- Unnamed: 0

X has feature names, but LogisticRegression was fitted without feature names


In [ ]:
accuracy_rf = accuracy_score(y,preds_rf)
roc_auc_rf =  roc_auc_score(y,preds_rf)
report_rf = classification_report(y, preds_rf, output_dict=True)

In [ ]:
question_no = 7
# perf_eval = Performance_Eval(model, 'Random Forest', X_train, y_train, X_valid, y_valid, question_no = question_no)
perf_eval = Performance_Eval(rf,'Random Forest',X_rf, preds_rf, X_rf, y, question_no = question_no)
roc = perf_eval.get_roc_auc_plot()
# experiment.log_figure(figure=roc,overwrite=False)
gr = perf_eval.get_goal_rate_plot()
# experiment.log_figure(figure=gr,overwrite=False)
cr = perf_eval.get_cum_rate_plot()
# experiment.log_figure(figure=cr,overwrite=False)
cp = perf_eval.get_calibration_plot()
# experiment.log_figure(figure=cp,overwrite=False)